<a href="https://colab.research.google.com/github/SreeBhargaviM/Neural-networks-with-Tensorflow/blob/master/Positive_negative_reviews_2layerLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv', engine='python', error_bad_lines=False)
test = pd.read_csv('test.csv', engine='python', error_bad_lines=False)

In [3]:
train.columns

Index(['employee ID', 'Place', 'location', 'date', 'status', 'job_title',
       'summary', 'positives', 'negatives', 'advice_to_mgmt', 'score_1',
       'score_2', 'score_3', 'score_4', 'score_5', 'score_6', 'overall'],
      dtype='object')

In [4]:
train_data = []
test_data = []

In [11]:
test['negatives'][100]

'Google only use its own tools, as a new-grad student, I do feel some pressure learning so many tools at the same time.'

In [15]:
def join_review(a,b):
  l = []
  for i in range(len(a)):
    inter = '' + str(a[i]) + '.' + ' ' + str(b[i])
    l.append(inter)
  return l

In [16]:
train_data = join_review(train['positives'],train['negatives'])
test_data = join_review(test['positives'],test['negatives'])

In [49]:
train_data[100]

'Possibly a growth company but financial problems create much uncertainty. I was hired through an Agency as a contract employee for 90 days, then offered a full-time position. This position lasted all of 24 days, then I was let go due to financial problems at the company. To make matters worse, I was told that I would receive severance on the forms I asked to sign. The company has reneged on that. I witnessed a parade of hiring/firing due to the lack of any business acumen, very poor communication and no evidence of a positive culture.'

In [22]:
train['overall'][0]

5

In [27]:
train_label = []

In [29]:
for i in range(len(train['overall'])):
  train_label.append(train['overall'][i])

In [30]:
train_label[10]

4

In [42]:
train_label = np.array(train_label)
train_label[0:10]

array([5, 5, 5, 4, 4, 5, 3, 5, 5, 5])

In [32]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [46]:
train_label = np.array(train_label)
train_label1 = tf.keras.utils.to_categorical(train_label)
train_label1[3]

0.0

In [47]:
n = np.shape(train_label1)
n

(60672, 6)

In [48]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [70]:
boring_words = ['in', 'a', 'the', 'for', 'and', 'with', 'an', 'was', 'that', 'this', 'them', 'they', 'at', 'of', 'as', 'I', 'to', 'told', 'would', 'could', 'due', 'then', 'any']

In [71]:
def filter_stop_words(train_sentences, stop_words):
    for i, sentence in enumerate(train_sentences):
        new_sent = [word for word in sentence.split() if word not in stop_words]
        train_sentences[i] = ' '.join(new_sent)
    return train_sentences

stop_words = boring_words
train_data = filter_stop_words(train_data, stop_words)

In [73]:
len(train_data[100])

418

In [74]:
train_data = np.array(train_data)

In [76]:
tokenizer = Tokenizer(num_words = 10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_data)
padded = pad_sequences(sequences,maxlen= 150, truncating= 'post')

testing_sequences = tokenizer.texts_to_sequences(test_data)
testing_padded = pad_sequences(testing_sequences,maxlen= 150)

In [77]:
x = np.shape(padded)
x

(30336, 150)

In [93]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 64, input_length=150),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    #tf.keras.layers.Dense(15, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 150, 64)           640000    
_________________________________________________________________
batch_normalization_5 (Batch (None, 150, 64)           256       
_________________________________________________________________
lstm_11 (LSTM)               (None, 150, 64)           33024     
_________________________________________________________________
lstm_12 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 198       
Total params: 685,894
Trainable params: 685,766
Non-trainable params: 128
_________________________________________________________________


In [94]:
model.fit(padded, train_label1, epochs=10, batch_size=45 ,validation_split=0.1)

Epoch 1/10
607/607 [==============================] - 108s 177ms/step - loss: 1.3730 - accuracy: 0.3580 - val_loss: 1.3451 - val_accuracy: 0.3780
Epoch 2/10
607/607 [==============================] - 109s 180ms/step - loss: 1.2703 - accuracy: 0.4106 - val_loss: 1.3488 - val_accuracy: 0.3593
Epoch 3/10
607/607 [==============================] - 109s 180ms/step - loss: 1.1193 - accuracy: 0.5141 - val_loss: 1.4438 - val_accuracy: 0.3395
Epoch 4/10
607/607 [==============================] - 109s 179ms/step - loss: 0.9128 - accuracy: 0.6241 - val_loss: 1.6240 - val_accuracy: 0.3378
Epoch 5/10
607/607 [==============================] - 111s 183ms/step - loss: 0.6863 - accuracy: 0.7335 - val_loss: 1.9204 - val_accuracy: 0.3210
Epoch 6/10
607/607 [==============================] - 109s 179ms/step - loss: 0.4992 - accuracy: 0.8144 - val_loss: 2.3048 - val_accuracy: 0.3171
Epoch 7/10
607/607 [==============================] - 109s 179ms/step - loss: 0.3607 - accuracy: 0.8688 - val_loss: 2.6205 -